In [ ]:
!pip install transformers
!pip install pip install tensorflow-addons

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
home_folder = Path('/content/drive/MyDrive/Colab Notebooks/humain')

In [ ]:
import pandas as pd
from pathlib import Path
from pickle import dump, load
from sklearn.model_selection import train_test_split
from pickle import load, dump

In [ ]:
# check source S of use cases
data_folder = Path('/content/drive/MyDrive/Colab Notebooks/humain/data')
usecases_a = pd.read_excel(data_folder / 'springboard_ai_usecases.xlsx')
usecases_a.dropna(axis=1, how='all', inplace=True)

In [ ]:
usecases_a.describe(include = 'all')

,Usecase Description,Type,Industry,Impact,Data Richness
count,121,121,121,121.000000,121.000000
unique,115,9,12,NaN,NaN
top,Predict lifetime value and risk of churn for i...,Predictive analytics,public and social sector,NaN,NaN
freq,4,24,11,NaN,NaN
mean,NaN,NaN,NaN,0.595868,0.815702
std,NaN,NaN,NaN,0.399457,0.499167
min,NaN,NaN,NaN,0.000000,0.000000
25%,NaN,NaN,NaN,0.300000,0.300000
50%,NaN,NaN,NaN,0.500000,0.700000
75%,NaN,NaN,NaN,0.800000,1.300000


In [ ]:
# inspection tells us all descriptions are example usecase sentences
usecases_a.head()

,Usecase Description,Type,Industry,Impact,Data Richness
0,Identify and navigate roads and obstructions i...,Process unstructured data,automotive,1.5,2.0
1,Predict failure and recommend proactive mainte...,Predictive maintenance,automotive,0.9,1.0
2,Optimize manufacturing process in real time— d...,Operations/logistics optimisation (real time),automotive,0.7,1.0
3,Optimize lane choices and path routing based o...,Operations/logistics optimisation (real time),automotive,0.6,1.7
4,Recognize complex voice commands to access gre...,Process unstructured data,automotive,0.5,0.3


In [ ]:
# The size of the available dataset is going to be a limiting factor in computing
# stable test scores. Between Type and Industry, type labels would create larger bins.
usecases_a_types = usecases_a.Type.unique()
usecases_a_types

array(['Process unstructured data', 'Predictive maintenance',
       'Operations/logistics optimisation (real time)', 'Forecasting',
       'Discover new trends/anomalies', 'Price and product optimisation',
       'Radical personalisation', 'Predictive analytics',
       'Resource allocation'], dtype=object)

In [ ]:
# load source B of use cases sentences
usecases_b = pd.read_json(data_folder / 'case_data.json', orient= 'split')
usecases_b.describe(include = 'all')

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: FutureWarning: Treating datetime data as categorical rather than numeric in `.describe` is deprecated and will be removed in a future version of pandas. Specify `datetime_is_numeric=True` to silence this warning and adopt the future behavior now.
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: FutureWarning: Treating datetime data as categorical rather than numeric in `.describe` is deprecated and will be removed in a future version of pandas. Specify `datetime_is_numeric=True` to silence this warning and adopt the future behavior now.
  This is separate from the ipykernel package so we can avoid doing imports until


,id,name,description,metrics,industries,created_at,updated_at,function
count,122.000000,122,122,122,122,122,122,122
unique,NaN,122,122,34,53,122,122,11
top,NaN,Personalized medications and care,Combine your data from different sources into ...,,,2020-02-03 15:08:21.886000+00:00,2020-02-03 15:08:21.886000+00:00,Marketing
freq,NaN,1,1,35,69,1,1,25
first,NaN,NaN,NaN,NaN,NaN,2020-02-03 15:08:21.159000+00:00,2020-02-03 15:08:21.159000+00:00,NaN
last,NaN,NaN,NaN,NaN,NaN,2020-02-03 15:08:23.511000+00:00,2020-02-03 15:08:23.511000+00:00,NaN
mean,554.336066,NaN,NaN,NaN,NaN,NaN,NaN,NaN
std,35.601934,NaN,NaN,NaN,NaN,NaN,NaN,NaN
min,493.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25%,524.250000,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
usecases_b.head()

,id,name,description,metrics,industries,created_at,updated_at,function
0,493,Retargeting,Retarget customers who have already expressed ...,Cash flow\nReturn on marketing investment (ROMI),Broadcasting\nFilm Distribution\nMedia and Ent...,2020-02-03 15:08:21.159000+00:00,2020-02-03 15:08:21.159000+00:00,Marketing
1,494,Recommendation engine,Also called recommendation personalization sys...,Cash flow\nReturn on marketing investment (ROM...,Search Engine\nRetail\nShopping\nE-Commerce\nL...,2020-02-03 15:08:21.212000+00:00,2020-02-03 15:08:21.212000+00:00,Marketing
2,495,Social analytics & automation,Leverage Natural Language Processing and machi...,Customer service effectiveness\nNet Promoter S...,Cloud Computing\nSoftware Solutions\nInformati...,2020-02-03 15:08:21.249000+00:00,2020-02-03 15:08:21.249000+00:00,Marketing
3,496,Product Information Management,Manage and improve all your product informatio...,Return on marketing investment (ROMI),Communications Infrastructure\nHardware\nEnter...,2020-02-03 15:08:21.270000+00:00,2020-02-03 15:08:21.270000+00:00,Marketing
4,497,Marketing analytics,Connect all your marketing data and KPIs autom...,Return on marketing investment (ROMI),Software Solutions\nCollaboration\nCRM\nData S...,2020-02-03 15:08:21.291000+00:00,2020-02-03 15:08:21.291000+00:00,Marketing


In [ ]:
for i in range(6):
    print(usecases_b.sample(1).description.values, '\n')

['Advanced analytics on call data to uncover insights to increase sales effectiveness'] 

['Leverage machine learning and big data to optimize your online or offline merchandising'] 

['Predictively maintain your robots and other machinery to minimize disruptions to operations'] 

['Automatically sync calendar, adressbook, emails, phone calls and messages of your salesforce to your CRM system. Enjoy better sales visibility and analytics while giving your sales personnel more sales time.'] 

['Analyze sound data for analytics and voice-controlled devices'] 

["Digitize your processes in weeks without replacing legacy systems which can take years. Bots can operate on legacy systems learning from your personnel's instructions and actions."] 



In [ ]:
# Use case sentence examples are found in the first and second sentence of the description
# when the description consists of maximum 2 sentences
# Eplode these first two sentences in rows

usecases_b['description_sentences'] = usecases_b['description'].str.split(r'\. ')
usecases_b_sel = usecases_b[~(usecases_b['description_sentences'].apply(len) > 2)]
usecases_b_sel = usecases_b_sel.explode('description_sentences')
usecases_b_sel.head()

,id,name,description,metrics,industries,created_at,updated_at,function,description_sentences
0,493,Retargeting,Retarget customers who have already expressed ...,Cash flow\nReturn on marketing investment (ROMI),Broadcasting\nFilm Distribution\nMedia and Ent...,2020-02-03 15:08:21.159000+00:00,2020-02-03 15:08:21.159000+00:00,Marketing,Retarget customers who have already expressed ...
1,494,Recommendation engine,Also called recommendation personalization sys...,Cash flow\nReturn on marketing investment (ROM...,Search Engine\nRetail\nShopping\nE-Commerce\nL...,2020-02-03 15:08:21.212000+00:00,2020-02-03 15:08:21.212000+00:00,Marketing,Also called recommendation personalization sys...
2,495,Social analytics & automation,Leverage Natural Language Processing and machi...,Customer service effectiveness\nNet Promoter S...,Cloud Computing\nSoftware Solutions\nInformati...,2020-02-03 15:08:21.249000+00:00,2020-02-03 15:08:21.249000+00:00,Marketing,Leverage Natural Language Processing and machi...
3,496,Product Information Management,Manage and improve all your product informatio...,Return on marketing investment (ROMI),Communications Infrastructure\nHardware\nEnter...,2020-02-03 15:08:21.270000+00:00,2020-02-03 15:08:21.270000+00:00,Marketing,Manage and improve all your product informatio...
7,500,Shelf audit/analytics,"Use video, images or robots on the retail area...",B2C sales,Delivery\nManufacturing\nFood and Beverage\nOu...,2020-02-03 15:08:21.360000+00:00,2020-02-03 15:08:21.360000+00:00,Marketing,"Use video, images or robots on the retail area..."


In [ ]:
# list unique 'function' labels in source B along unique 'Type' labels in source a,
# to see if they have similar meaning
print(usecases_a_types)
print(usecases_b.function.unique())

['Process unstructured data' 'Predictive maintenance'
 'Operations/logistics optimisation (real time)' 'Forecasting'
 'Discover new trends/anomalies' 'Price and product optimisation'
 'Radical personalisation' 'Predictive analytics' 'Resource allocation']
['Marketing' 'Sales' 'IT' 'Customer Service' 'Operations' 'Data'
 'Analytics' 'Fintech' 'HR' 'Healthtech' 'Self driving cars']


In [ ]:
# There looks to be no easy way to match them.
# I decide to manually tag all source B use case sentences with 'Type' labels
usecases_b_sel.description_sentences.to_csv(home_folder / 'case_data_usecases_unlabeled.csv')

In [ ]:
# Load manually labeled cases from source B
usecases_b_labeled = pd.read_csv(home_folder / 'case_data_usecases_labeled.csv',
                                header = None)
usecases_b_labeled.rename({0: 'Type', 1: 'Usecase Description'},
                          axis = 1,
                          inplace=True)
usecases_b_labeled.head()

,Type,Usecase Description
0,Radical personalisation,Retarget customers who have already expressed ...
1,Radical personalisation,Also called recommendation personalization sys...
2,Process unstructured data,Leverage Natural Language Processing and machi...
3,Price and product optimisation,Manage and improve all your product informatio...
4,Process unstructured data,"Use video, images or robots on the retail area..."


In [ ]:
# concatenate with source A and check the typical lenght of use cases
usecases = pd.concat([usecases_b_labeled, usecases_a], join='inner')
usecases['Usecase Description'].str.len().describe()

count    228.000000
mean      96.719298
std       35.005351
min       36.000000
25%       70.000000
50%       91.000000
75%      117.250000
max      219.000000
Name: Usecase Description, dtype: float64

In [ ]:
# balance the dataset with an equal number of verified non-usecase sentences
random_sentences = pd.read_csv(home_folder / 'random_report_text.txt',
                               header = None,
                              sep = r'\. ')
random_sentences = random_sentences.transpose()
random_sentences.to_csv(home_folder / 'random_report_sentences_candidates.csv')


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  """


In [ ]:
# load back the verified list of non-usecase sentences and check the typical length
# they seem to be a bit longer but close enough to not just be able to decide based
# on lenght alone
report_sentences_sel = pd.read_csv(home_folder / 'random_report_sentences_selection.csv',
                                   header = None)
report_sentences_sel.rename({0: 'Usecase Description'},
                            axis = 1,
                            inplace=True)
report_sentences_sel['Usecase Description'].str.len().describe()

count    220.000000
mean     118.909091
std       29.329352
min       50.000000
25%       99.750000
50%      119.000000
75%      139.000000
max      202.000000
Name: Usecase Description, dtype: float64

In [ ]:
# create the dataset of usecases and no usecases
dataset = pd.concat([usecases, report_sentences_sel])
dataset.sample(5)

,Type,Usecase Description
12,NaN,Understanding the contextual landscape around ...
57,NaN,Some cities have also established real-time da...
167,NaN,Other domains could be fertile ground for brin...
108,NaN,Data and analytics underpin several disruptive...
109,NaN,Introducing new types of data sets (“orthogona...


In [ ]:
# do some specific sentence cleaning

import re

def clean(sequence):
    # remove the sequence ending punctuation because source_a doesn't have it
    sequence = sequence.replace(r'(\.|\?|\!|\:|\,)$', '')
    # remove 2 or more consecutive spaces with just one
    sequence = re.sub(" +", " ", sequence)
    # remove leading and trailing whitespaces (space and line breaks)
    sequence = sequence.strip()
    # convert to lowercase
    sequence = sequence.lower()
    # get first 256 words
    sequence = sequence.split(maxsplit = 256)[:256] # get list of first 256 words
    sequence = ' '.join(sequence)
    
    return sequence

dataset['Usecase Description'] = dataset['Usecase Description'].apply(func = clean)
for i in range(6):
    print(dataset['Usecase Description'].sample(1).values, '\n')

['advanced analytics on all sales call data to uncover insights to increase sales effectiveness'] 

['predict risk of illicit activity or terrorism using historical crime data, intelligence data, and other available sources (e.g., predictive policing)'] 

['leverage natural language processing to monitor social media to make real-time business decisions'] 

['this has enabled basic analytics but little has been done to unlock and fully utilize the vast stores of data actually contained within emrs'] 

['most of the relevant regulations relate to privacy and ownership of personally identifiable information'] 

['but another equally vital role is that of the business translator who serves as the link between analytical talent and practical applications to business questions'] 



In [ ]:
# Create codes for a use case / no use case label
dataset['usecase'] = pd.Categorical(~dataset['Type'].isnull())
dataset['usecase'] = dataset['usecase'].cat.codes
dataset.sample(5)

,Type,Usecase Description,usecase
21,Radical personalisation,personalize product recommendations and advert...,1
32,Resource allocation,use bots on your retail floor to answer custom...,1
130,NaN,our 2011 report analyzed how the european unio...,0
67,Predictive analytics,evaluate doctor performance and provide outcom...,1
81,Operations/logistics optimisation (real time),put in place control mechanisms that detect an...,1


In [ ]:
# Create codes for a type label
dataset['Type'] = dataset['Type'].fillna('no use case')
dataset['Type_cat'] = pd.Categorical(dataset['Type'])
dataset['type'] = dataset['Type_cat'].cat.codes

pickle_folder = Path(home_folder / 'pickle')

d = dict(enumerate(dataset['Type_cat'].cat.categories))
print(d)

with open(pickle_folder / 'type_dictionary.pkl','wb') as f:
  dump(d, f)
dataset.head(5)

{0: 'Discover new trends/anomalies', 1: 'Forecasting', 2: 'Operations/logistics optimisation (real time)', 3: 'Predictive analytics', 4: 'Predictive maintenance', 5: 'Price and product optimisation', 6: 'Process unstructured data', 7: 'Radical personalisation', 8: 'Resource allocation', 9: 'no use case'}


,Type,Usecase Description,usecase,Type_cat,type
0,Radical personalisation,retarget customers who have already expressed ...,1,Radical personalisation,7
1,Radical personalisation,also called recommendation personalization sys...,1,Radical personalisation,7
2,Process unstructured data,leverage natural language processing and machi...,1,Process unstructured data,6
3,Price and product optimisation,manage and improve all your product informatio...,1,Price and product optimisation,5
4,Process unstructured data,"use video, images or robots on the retail area...",1,Process unstructured data,6


In [ ]:
# See the distribution of labels
dataset[['Type', 'usecase']].groupby(['usecase', 'Type']).size()

usecase  Type                                         
0        no use case                                      220
1        Discover new trends/anomalies                     12
         Forecasting                                       18
         Operations/logistics optimisation (real time)     21
         Predictive analytics                              55
         Predictive maintenance                            10
         Price and product optimisation                    22
         Process unstructured data                         46
         Radical personalisation                           20
         Resource allocation                               24
dtype: int64

In [ ]:
# split the dataset in train and validation
from sklearn.model_selection import train_test_split

train, val = train_test_split(dataset[['Usecase Description', 'usecase', 'type']],
                              test_size = 0.2,
                              stratify = dataset['type'])

print('train type:', type(train))
print('train shape:', train.shape)

print('val type:', type(val))
print('val shape:', val.shape)

train type: <class 'pandas.core.frame.DataFrame'>
train shape: (358, 3)
val type: <class 'pandas.core.frame.DataFrame'>
val shape: (90, 3)


In [ ]:
# load the tokenizer
from transformers import BertTokenizer, BertConfig

config = BertConfig.from_pretrained('bert-base-uncased')
config.output_hidden_states = False
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', config = config)

In [ ]:
# tokenize the sentences
# we don't truncate any sentence in the dataset by selecting max_length 40
max_length = 40

encoded_data_train = tokenizer(list(train['Usecase Description']),
                               max_length = max_length,
                               truncation = True,
                               add_special_tokens = True,
                               return_tensors = 'tf',
                               padding = 'max_length',
                               return_attention_mask = True,
                               return_token_type_ids = False,
                               verbose = True)

encoded_data_val = tokenizer(list(val['Usecase Description']),
                               max_length = max_length,
                               truncation = True,
                               add_special_tokens = True,
                               return_tensors = 'tf',
                               padding = 'max_length',
                               return_attention_mask = True,
                               return_token_type_ids = False,
                               verbose = True)

print(encoded_data_train)
print(encoded_data_val)

{'input_ids': <tf.Tensor: shape=(358, 40), dtype=int32, numpy=
array([[  101,  2023,  9575, ...,     0,     0,     0],
       [  101,  3935,  2966, ...,     0,     0,     0],
       [  101,  4283,  2000, ...,     0,     0,     0],
       ...,
       [  101,  6133,  1998, ...,     0,     0,     0],
       [  101, 16014,  4945, ...,     0,     0,     0],
       [  101, 23569, 27605, ...,     0,     0,     0]], dtype=int32)>, 'attention_mask': <tf.Tensor: shape=(358, 40), dtype=int32, numpy=
array([[1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       ...,
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0]], dtype=int32)>}
{'input_ids': <tf.Tensor: shape=(90, 40), dtype=int32, numpy=
array([[  101,  8073, 26351, ...,     0,     0,     0],
       [  101,  2224,  7976, ...,     0,     0,     0],
       [  101,  1996,  3643, ...,     0,     0,     0],
       ...,
       [  101, 23569, 27605, ...,     0,    

In [ ]:
# one-hot label creation for type
from tensorflow.keras.utils import to_categorical

type_vec_train = to_categorical(train['type'])
print(type_vec_train[0])
print(type_vec_train.shape)

type_vec_val = to_categorical(val['type'])
print(type_vec_val[0])
print(type_vec_val.shape)

[0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
(358, 10)
[0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
(90, 10)


In [ ]:
# load a BERT model

from transformers import TFBertModel

model = TFBertModel.from_pretrained('bert-base-uncased', config = config)
display(model.summary())

Some layers from the model checkpoint at bert-base-uncased were not used when initializing TFBertModel: ['nsp___cls', 'mlm___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


Model: "tf_bert_model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bert (TFBertMainLayer)       multiple                  109482240 
Total params: 109,482,240
Trainable params: 109,482,240
Non-trainable params: 0
_________________________________________________________________


None

In [ ]:
# Adapt the model for multiclass and multilabel classification

from tensorflow.keras.layers import Input, Dropout, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.initializers import TruncatedNormal

#######################################   
### ------- Build the model ------- ###
# TF Keras documentation: https://www.tensorflow.org/api_docs/python/tf/keras/Model# Load the MainLayer   
bert = model.layers[0]
# Build your model input   
input_ids = Input(shape=(max_length,), name='input_ids', dtype='int32')
attention_mask = Input(shape=(max_length,), name='attention_mask', dtype='int32')
inputs = {'input_ids': input_ids, 'attention_mask': attention_mask}
# Load the Transformers BERT model as a layer in a Keras model   
bert_model = bert(inputs)[1]
dropout = Dropout(config.hidden_dropout_prob, name='pooled_output')   
pooled_output = dropout(bert_model, training=False)
# Then build your model output   
usecase = Dense(units=1, kernel_initializer=TruncatedNormal(stddev=config.initializer_range), name='usecase')(pooled_output)
type_ = Dense(units=len(dataset.type.value_counts()), kernel_initializer=TruncatedNormal(stddev=config.initializer_range), name='type')(pooled_output)
outputs = {'usecase': usecase, 'type': type_}
# And combine it all in a model object   
model = Model(inputs=inputs, outputs=outputs, name='BERT_MultiClass_MultiLabel')
# Take a look at the model   
model.summary()

Model: "BERT_MultiClass_MultiLabel"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
attention_mask (InputLayer)     [(None, 40)]         0                                            
__________________________________________________________________________________________________
input_ids (InputLayer)          [(None, 40)]         0                                            
__________________________________________________________________________________________________
bert (TFBertMainLayer)          TFBaseModelOutputWit 109482240   attention_mask[0][0]             
                                                                 input_ids[0][0]                  
__________________________________________________________________________________________________
pooled_output (Dropout)         (None, 768)          0           bert[0][

In [ ]:
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.losses import CategoricalCrossentropy, BinaryCrossentropy
from tensorflow.keras.metrics import CategoricalAccuracy
import tensorflow_addons as tfa

batch_size = 64

#######################################
### ------- Train the model ------- ###
# Set an optimizer
optimizer = Adam(   
    learning_rate=1e-05,   
    epsilon=1e-08,   
    decay=0.01,   
    clipnorm=1.0)
# Set loss and metrics
loss = {'usecase': BinaryCrossentropy(from_logits = True),
        'type': CategoricalCrossentropy(from_logits = True)}
metric = {'usecase': ['accuracy',
                      tfa.metrics.F1Score(num_classes = 1,
                                          threshold = 0.5,
                                          average = 'macro')],
          'type': [CategoricalAccuracy('accuracy'),
                   tfa.metrics.F1Score(num_classes = 10, average = 'weighted')]}
# metric = {'usecase': ['accuracy'],
#           'type': [CategoricalAccuracy('accuracy')]}
# Compile the model
model.compile(
    optimizer = optimizer,
    loss = loss,
    metrics = metric)
# Ready output data for the model
y_usecase = train.usecase.to_numpy()
y_type = type_vec_train
# Tokenize the input (takes some time)
x = encoded_data_train
# Fit the model
history = model.fit(   
    x={'input_ids': x['input_ids'], 'attention_mask': x['attention_mask']},   
    y={'usecase': y_usecase, 'type': y_type},
    batch_size=batch_size,
    epochs=10)

Epoch 1/10
6/6 [==============================] - 26s 859ms/step - loss: 0.6549 - type_loss: 0.5977 - usecase_loss: 0.0572 - type_accuracy: 0.8978 - type_f1_score: 0.8738 - usecase_accuracy: 1.0000 - usecase_f1_score: 1.0000
Epoch 2/10
6/6 [==============================] - 5s 845ms/step - loss: 0.6261 - type_loss: 0.5627 - usecase_loss: 0.0634 - type_accuracy: 0.8701 - type_f1_score: 0.8428 - usecase_accuracy: 1.0000 - usecase_f1_score: 1.0000
Epoch 3/10
6/6 [==============================] - 5s 851ms/step - loss: 0.6356 - type_loss: 0.5807 - usecase_loss: 0.0549 - type_accuracy: 0.8778 - type_f1_score: 0.8440 - usecase_accuracy: 1.0000 - usecase_f1_score: 1.0000
Epoch 4/10
6/6 [==============================] - 5s 848ms/step - loss: 0.5981 - type_loss: 0.5429 - usecase_loss: 0.0552 - type_accuracy: 0.9121 - type_f1_score: 0.8883 - usecase_accuracy: 1.0000 - usecase_f1_score: 1.0000
Epoch 5/10
6/6 [==============================] - 5s 850ms/step - loss: 0.5533 - type_loss: 0.5010 - us

In [ ]:
# It only requires light training to get good accuracy of usecase/no usecase
####################################### 
### ----- Evaluate the model ------ ###
# Ready test data   
val_y_usecase = val.usecase.to_numpy()
val_y_type = type_vec_val
val_x = encoded_data_val
# Run evaluation   
model_eval = model.evaluate(   
    x={'input_ids': val_x['input_ids'], 'attention_mask': val_x['attention_mask']},   
    y={'usecase': val_y_usecase, 'type': val_y_type},
    verbose = 2
)

3/3 - 1s - loss: 0.9102 - type_loss: 0.7760 - usecase_loss: 0.1342 - type_accuracy: 0.8000 - type_f1_score: 0.5999 - usecase_accuracy: 0.9667 - usecase_f1_score: 0.9663


In [ ]:
# After ~70 epochs these results still look good only for the usecase (anything besides 9)
# or no usecase (9) decision. But in reality we found the results of the lightly trained
# model better.
from sklearn.metrics import classification_report
import numpy as np

# Convert one-hot to index
y_type_val = np.argmax(type_vec_val, axis=1)
y_pred = model.predict(x={'input_ids': encoded_data_val['input_ids'], 'attention_mask': encoded_data_val['attention_mask']})
y_type_pred = np.argmax(y_pred['type'],axis=1)
print(classification_report(y_type_val, y_type_pred))

              precision    recall  f1-score   support

           0       0.50      0.50      0.50         2
           1       0.40      0.50      0.44         4
           2       1.00      0.25      0.40         4
           3       0.45      0.45      0.45        11
           4       0.67      1.00      0.80         2
           5       0.50      0.60      0.55         5
           6       0.75      0.33      0.46         9
           7       1.00      1.00      1.00         4
           8       0.25      0.40      0.31         5
           9       0.96      1.00      0.98        44

    accuracy                           0.74        90
   macro avg       0.65      0.60      0.59        90
weighted avg       0.77      0.74      0.74        90



In [ ]:
# only save when the current model is outperforming the previous one
model.save_weights("ckpt")

In [ ]:
# make it smaller by not compiling (loading weights only) and resaving as h5?
# FIRST RECREATE MODEL FROM SCRATCH without compilation above
model.load_weights("ckpt")
model.save(home_folder / 'usecase_indicator.h5')

In [ ]:
from tensorflow import keras
import tensorflow_addons as tfa
import numpy as np
from sklearn.metrics import classification_report

model = keras.models.load_model(home_folder / 'usecase_indicator.h5')

# Convert one-hot to index
y_type_val = np.argmax(type_vec_val, axis=1)
y_pred = model.predict(x={'input_ids': encoded_data_val['input_ids'], 'attention_mask': encoded_data_val['attention_mask']})
y_type_pred = np.argmax(y_pred['type'],axis=1)
print(classification_report(y_type_val, y_type_pred, target_names = [value for value in d.values()]))

                                               precision    recall  f1-score   support

                Discover new trends/anomalies       0.00      0.00      0.00         2
                                  Forecasting       0.00      0.00      0.00         4
Operations/logistics optimisation (real time)       0.00      0.00      0.00         4
                         Predictive analytics       0.23      0.82      0.35        11
                       Predictive maintenance       0.00      0.00      0.00         2
               Price and product optimisation       0.00      0.00      0.00         5
                    Process unstructured data       0.00      0.00      0.00         9
                      Radical personalisation       0.00      0.00      0.00         4
                          Resource allocation       0.00      0.00      0.00         5
                                  no use case       0.92      1.00      0.96        44

                                     accu

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
